<a href="https://colab.research.google.com/github/chhezue/NH-Invest-BigData/blob/main/3_Final_ETF_Selection_By_Intersection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from scipy.stats import pearsonr, spearmanr
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df_etf_final = pd.read_csv('/content/gdrive/MyDrive/NH 공모전/data/df_etf_final.csv')
df_etf_final

,tck_iem_cd,Week,yield_rate,금_주차,금_증감률,코인_증감률,환율_증감률
0,AAN,Week: 2024-05-27/2024-06-02,17.61,NaN,NaN,NaN,NaN
1,AAN,Week: 2024-06-03/2024-06-09,-7.28,Week: 2024-05-27/2024-06-02,1.42,-1.11,0.81
2,AAN,Week: 2024-06-10/2024-06-16,-5.28,Week: 2024-06-03/2024-06-09,1.25,2.79,-0.73
3,AAN,Week: 2024-06-17/2024-06-23,-0.70,Week: 2024-06-10/2024-06-16,0.33,-4.32,0.30
4,AAN,Week: 2024-06-24/2024-06-30,0.81,Week: 2024-06-17/2024-06-23,2.05,-5.20,0.44
...,...,...,...,...,...,...,...
22790,ZYXI,Week: 2024-07-29/2024-08-04,-6.18,Week: 2024-07-22/2024-07-28,-1.56,0.16,-0.09
22791,ZYXI,Week: 2024-08-05/2024-08-11,-5.36,Week: 2024-07-29/2024-08-04,2.83,-14.87,-1.41
22792,ZYXI,Week: 2024-08-12/2024-08-18,2.37,Week: 2024-08-05/2024-08-11,-0.95,1.05,0.36
22793,ZYXI,Week: 2024-08-19/2024-08-25,5.81,Week: 2024-08-12/2024-08-18,-0.72,-0.41,-0.58


# 증감률에 따라 대폭/소폭 주 나누기
* 금: 1.42% 기준으로 분리(2.83%의 50%)
* 코인: 6.04% 기준으로 분리(12.07%의 50%)
* 환율: 0.40% 기준으로 분리(0.81%의 50%)

In [ ]:
print(f'금_증감률 max: {df_etf_final["금_증감률"].max()}, min: {df_etf_final["금_증감률"].min()}')
print(f'코인_증감률 max: {df_etf_final["코인_증감률"].max()}, min: {df_etf_final["코인_증감률"].min()}')
print(f'환율_증감률 max: {df_etf_final["환율_증감률"].max()}, min: {df_etf_final["환율_증감률"].min()}')

금_증감률 max: 2.83, min: -1.56
코인_증감률 max: 12.07, min: -14.87
환율_증감률 max: 0.81, min: -4.0


In [ ]:
def label_gold(low):
    if pd.isna(low['금_증감률']):
        return None
    elif low['금_증감률'] >= 1.42:
        return "대폭"
    else:
        return "소폭"

def label_coin(low):
    if pd.isna(low['코인_증감률']):
        return None
    elif low['코인_증감률'] >= 6.04:
        return "대폭"
    else:
        return "소폭"

def label_exchange(low):
    if pd.isna(low['환율_증감률']):
        return None
    elif low['환율_증감률'] >= 0.40:
        return "대폭"
    else:
        return "소폭"

# 각 자산에 대해 라벨링
df_etf_final['금_라벨'] = df_etf_final.apply(label_gold, axis=1)
df_etf_final['코인_라벨'] = df_etf_final.apply(label_coin, axis=1)
df_etf_final['환율_라벨'] = df_etf_final.apply(label_exchange, axis=1)

In [ ]:
df_etf_final = df_etf_final[['tck_iem_cd', 'Week', 'yield_rate', '금_주차', '금_증감률', '금_라벨', '코인_증감률', '코인_라벨', '환율_증감률', '환율_라벨']]
df_etf_final

,tck_iem_cd,Week,yield_rate,금_주차,금_증감률,금_라벨,코인_증감률,코인_라벨,환율_증감률,환율_라벨
0,AAN,Week: 2024-05-27/2024-06-02,17.61,NaN,NaN,None,NaN,None,NaN,None
1,AAN,Week: 2024-06-03/2024-06-09,-7.28,Week: 2024-05-27/2024-06-02,1.42,대폭,-1.11,소폭,0.81,대폭
2,AAN,Week: 2024-06-10/2024-06-16,-5.28,Week: 2024-06-03/2024-06-09,1.25,소폭,2.79,소폭,-0.73,소폭
3,AAN,Week: 2024-06-17/2024-06-23,-0.70,Week: 2024-06-10/2024-06-16,0.33,소폭,-4.32,소폭,0.30,소폭
4,AAN,Week: 2024-06-24/2024-06-30,0.81,Week: 2024-06-17/2024-06-23,2.05,대폭,-5.20,소폭,0.44,대폭
...,...,...,...,...,...,...,...,...,...,...
22790,ZYXI,Week: 2024-07-29/2024-08-04,-6.18,Week: 2024-07-22/2024-07-28,-1.56,소폭,0.16,소폭,-0.09,소폭
22791,ZYXI,Week: 2024-08-05/2024-08-11,-5.36,Week: 2024-07-29/2024-08-04,2.83,대폭,-14.87,소폭,-1.41,소폭
22792,ZYXI,Week: 2024-08-12/2024-08-18,2.37,Week: 2024-08-05/2024-08-11,-0.95,소폭,1.05,소폭,0.36,소폭
22793,ZYXI,Week: 2024-08-19/2024-08-25,5.81,Week: 2024-08-12/2024-08-18,-0.72,소폭,-0.41,소폭,-0.58,소폭


In [ ]:
import pandas as pd
from scipy.stats import pearsonr, spearmanr

# 분석할 변수 목록
variables = ['금', '코인', '환율']

# 결과를 저장할 딕셔너리 초기화
results_dict = {variable: {} for variable in variables}

# 각 종목에 대해 반복
for ticker in df_etf_final['tck_iem_cd'].unique():
    sample = df_etf_final[df_etf_final['tck_iem_cd'] == ticker]

    # 각 변수에 대해 반복
    for variable in variables:
        label_column = f'{variable}_라벨'
        change_rate_column = f'{variable}_증감률'

        # 소폭 및 대폭 데이터프레임 생성
        low_df = sample[sample[label_column] == '소폭'].sort_values(by=change_rate_column)
        high_df = sample[sample[label_column] == '대폭'].sort_values(by=change_rate_column)

        # 소폭에 대한 피어슨 및 스피어만 상관계수 계산
        if low_df.shape[0] >= 2:
            pearson_low = pearsonr(low_df['yield_rate'], low_df[change_rate_column].dropna())[0]
            spearman_low = spearmanr(low_df['yield_rate'], low_df[change_rate_column].dropna())[0]
        else:
            pearson_low = None
            spearman_low = None

        # 대폭에 대한 피어슨 및 스피어만 상관계수 계산
        if high_df.shape[0] >= 2:
            pearson_high = pearsonr(high_df['yield_rate'], high_df[change_rate_column].dropna())[0]
            spearman_high = spearmanr(high_df['yield_rate'], high_df[change_rate_column].dropna())[0]
        else:
            pearson_high = None
            spearman_high = None

        # 결과를 딕셔너리에 추가
        if variable not in results_dict:
            results_dict[variable] = {}

        if ticker not in results_dict[variable]:
            results_dict[variable][ticker] = {}

        results_dict[variable][ticker] = {
            'tck_iem_cd': ticker,
            f'{variable}_소_피어슨': pearson_low,
            f'{variable}_소_스피어만': spearman_low,
            f'{variable}_대_피어슨': pearson_high,
            f'{variable}_대_스피어만': spearman_high
        }

# 각 변수별 데이터프레임 생성
results_dfs = {variable: pd.DataFrame.from_dict(results_dict[variable], orient='index') for variable in variables}

<ipython-input-6-6a64b1b210c2>:33: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  pearson_high = pearsonr(high_df['yield_rate'], high_df[change_rate_column].dropna())[0]
<ipython-input-6-6a64b1b210c2>:34: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_high = spearmanr(high_df['yield_rate'], high_df[change_rate_column].dropna())[0]


In [ ]:
gold_df = results_dfs['금']
coin_df = results_dfs['코인']
exchange_df = results_dfs['환율']

print(gold_df)
print(coin_df)
print(exchange_df)

     tck_iem_cd   금_소_피어슨  금_소_스피어만   금_대_피어슨  금_대_스피어만
AAN         AAN -0.661378 -0.728040  0.572818       0.8
AAON       AAON  0.177372  0.300000  0.561124       0.8
AAPB       AAPB -0.273125 -0.316667  0.880535       0.2
AAT         AAT  0.127686  0.283333 -0.085943       0.2
ABCB       ABCB  0.530589  0.400000  0.190518       0.2
...         ...       ...       ...       ...       ...
ZUMZ       ZUMZ -0.062324 -0.166667  0.244698       0.4
ZURA       ZURA  0.073128  0.050000  0.464929       0.2
ZVRA       ZVRA  0.599648  0.416667 -0.643747      -0.4
ZYME       ZYME -0.028913  0.100000 -0.387906       0.0
ZYXI       ZYXI -0.093530 -0.066667 -0.362097       0.2

[1629 rows x 5 columns]
     tck_iem_cd  코인_소_피어슨  코인_소_스피어만  코인_대_피어슨  코인_대_스피어만
AAN         AAN -0.374965  -0.272727  0.937427        1.0
AAON       AAON -0.268764  -0.127273  0.951952        1.0
AAPB       AAPB -0.139235   0.030303  0.015072        0.5
AAT         AAT -0.343386  -0.090909  0.806425        0.5
ABCB       AB

# 상관계수 0.3

In [ ]:
def add_corr(df, variable, pearson_threshold=0.3, spearman_threshold=0.3):
    # 소폭 상관관계 추가
    df[f'{variable}_소_양_상관관계'] = ((df[f'{variable}_소_피어슨'] >= pearson_threshold) &
                                         (df[f'{variable}_소_스피어만'] >= spearman_threshold)).astype(int)
    df[f'{variable}_소_음_상관관계'] = ((df[f'{variable}_소_피어슨'] <= -pearson_threshold) &
                                         (df[f'{variable}_소_스피어만'] <= -spearman_threshold)).astype(int)

    # 대폭 상관관계 추가
    df[f'{variable}_대_양_상관관계'] = ((df[f'{variable}_대_피어슨'] >= pearson_threshold) &
                                         (df[f'{variable}_대_스피어만'] >= spearman_threshold)).astype(int)
    df[f'{variable}_대_음_상관관계'] = ((df[f'{variable}_대_피어슨'] <= -pearson_threshold) &
                                         (df[f'{variable}_대_스피어만'] <= -spearman_threshold)).astype(int)

# 소폭 및 대폭 상관관계 추가
add_corr(gold_df, '금', pearson_threshold=0.3, spearman_threshold=0.3)
add_corr(coin_df, '코인', pearson_threshold=0.3, spearman_threshold=0.3)
add_corr(exchange_df, '환율', pearson_threshold=0.3, spearman_threshold=0.3)

# 결과 확인
print(gold_df)
print(coin_df)
print(exchange_df)

     tck_iem_cd   금_소_피어슨  금_소_스피어만   금_대_피어슨  금_대_스피어만  금_소_양_상관관계  \
AAN         AAN -0.661378 -0.728040  0.572818       0.8           0   
AAON       AAON  0.177372  0.300000  0.561124       0.8           0   
AAPB       AAPB -0.273125 -0.316667  0.880535       0.2           0   
AAT         AAT  0.127686  0.283333 -0.085943       0.2           0   
ABCB       ABCB  0.530589  0.400000  0.190518       0.2           1   
...         ...       ...       ...       ...       ...         ...   
ZUMZ       ZUMZ -0.062324 -0.166667  0.244698       0.4           0   
ZURA       ZURA  0.073128  0.050000  0.464929       0.2           0   
ZVRA       ZVRA  0.599648  0.416667 -0.643747      -0.4           1   
ZYME       ZYME -0.028913  0.100000 -0.387906       0.0           0   
ZYXI       ZYXI -0.093530 -0.066667 -0.362097       0.2           0   

      금_소_음_상관관계  금_대_양_상관관계  금_대_음_상관관계  
AAN            1           1           0  
AAON           0           1           0  
AAPB           0  

In [ ]:
# 종목별 상관관계에 따라 필터링 및 리스트 생성
def get_related_names(corr_df, col, threshold):
    results = {}
    low_pos_related = corr_df[corr_df[f'{col}_소_양_상관관계'] == 1]['tck_iem_cd'].tolist()
    low_neg_related = corr_df[corr_df[f'{col}_소_음_상관관계'] == 1]['tck_iem_cd'].tolist()
    high_pos_related = corr_df[corr_df[f'{col}_대_양_상관관계'] == 1]['tck_iem_cd'].tolist()
    high_neg_related = corr_df[corr_df[f'{col}_대_음_상관관계'] == 1]['tck_iem_cd'].tolist()

    results[f'{col}_소_양_{threshold}'] = low_pos_related
    results[f'{col}_소_음_{threshold}'] = low_neg_related
    results[f'{col}_대_양_{threshold}'] = high_pos_related
    results[f'{col}_대_음_{threshold}'] = high_neg_related

    return results

# 상관관계 분석할 데이터프레임 목록
dataframes = {'금': gold_df, '코인': coin_df, '환율': exchange_df}

# 결과 저장을 위한 딕셔너리
related_names_all_3 = {}

# 각 데이터프레임에 대해 반복하여 관련 종목 리스트 생성
for key, df in dataframes.items():
    related_names_3 = get_related_names(df, key, threshold=3)
    related_names_all_3.update(related_names_3)  # 3의 결과를 업데이트

# 결과를 데이터프레임으로 변환
df_3 = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in related_names_all_3.items()]))
df_3

,금_소_양_3,금_소_음_3,금_대_양_3,금_대_음_3,코인_소_양_3,코인_소_음_3,코인_대_양_3,코인_대_음_3,환율_소_양_3,환율_소_음_3,환율_대_양_3,환율_대_음_3
0,ABCB,AAN,AAN,ABM,ACRS,ABCB,AAN,ABM,AFRI,ABG,AAPB,AAN
1,ABG,AGGH,AAON,ACHV,AGGH,ABG,AAON,AC,AMPL,ABOS,ABM,AAON
2,ABOS,AGR,AC,ACRE,AILE,ACEL,AAT,ACNB,BRC,ACA,AC,AAT
3,AC,AMDY,ACA,ACRS,BUCK,ACIU,ABG,ACRE,CTRI,ACLS,ACEL,ABCB
4,ACA,APLY,ACCO,AGEN,CCRN,ACLS,ABOS,ADVM,ENV,ACLX,ACHV,ABG
...,...,...,...,...,...,...,...,...,...,...,...,...
1231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZIP
1232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZUMZ
1233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZURA
1234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZVRA


In [ ]:
def df_count(df):
    for column in df.columns:
        unique_items = df[column].dropna().unique()
        count = len(unique_items)
        print(f"{column} 개수: {count}")

df_count(df_3)

금_소_양_3 개수: 560
금_소_음_3 개수: 100
금_대_양_3 개수: 595
금_대_음_3 개수: 256
코인_소_양_3 개수: 39
코인_소_음_3 개수: 634
코인_대_양_3 개수: 988
코인_대_음_3 개수: 345
환율_소_양_3 개수: 26
환율_소_음_3 개수: 206
환율_대_양_3 개수: 390
환율_대_음_3 개수: 1236


# 상관계수 0.5

In [ ]:
add_corr(gold_df, '금', pearson_threshold=0.5, spearman_threshold=0.5)
add_corr(coin_df, '코인', pearson_threshold=0.5, spearman_threshold=0.5)
add_corr(exchange_df, '환율', pearson_threshold=0.5, spearman_threshold=0.5)

# 결과 확인
print(gold_df)
print(coin_df)
print(exchange_df)

     tck_iem_cd   금_소_피어슨  금_소_스피어만   금_대_피어슨  금_대_스피어만  금_소_양_상관관계  \
AAN         AAN -0.661378 -0.728040  0.572818       0.8           0   
AAON       AAON  0.177372  0.300000  0.561124       0.8           0   
AAPB       AAPB -0.273125 -0.316667  0.880535       0.2           0   
AAT         AAT  0.127686  0.283333 -0.085943       0.2           0   
ABCB       ABCB  0.530589  0.400000  0.190518       0.2           0   
...         ...       ...       ...       ...       ...         ...   
ZUMZ       ZUMZ -0.062324 -0.166667  0.244698       0.4           0   
ZURA       ZURA  0.073128  0.050000  0.464929       0.2           0   
ZVRA       ZVRA  0.599648  0.416667 -0.643747      -0.4           0   
ZYME       ZYME -0.028913  0.100000 -0.387906       0.0           0   
ZYXI       ZYXI -0.093530 -0.066667 -0.362097       0.2           0   

      금_소_음_상관관계  금_대_양_상관관계  금_대_음_상관관계  
AAN            1           1           0  
AAON           0           1           0  
AAPB           0  

In [ ]:
# 상관관계 분석할 데이터프레임 목록
dataframes = {'금': gold_df, '코인': coin_df, '환율': exchange_df}

# 결과 저장을 위한 딕셔너리
related_names_all_5 = {}

# 각 데이터프레임에 대해 반복하여 관련 종목 리스트 생성
for key, df in dataframes.items():
    related_names_5 = get_related_names(df, key, threshold=5)
    related_names_all_5.update(related_names_5)  # 5의 결과를 업데이트

# 결과를 데이터프레임으로 변환
df_5 = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in related_names_all_5.items()]))
df_5

,금_소_양_5,금_소_음_5,금_대_양_5,금_대_음_5,코인_소_양_5,코인_소_음_5,코인_대_양_5,코인_대_음_5,환율_소_양_5,환율_소_음_5,환율_대_양_5,환율_대_음_5
0,ACRE,AAN,AAN,ABM,AGGH,ACEL,AAN,AC,MRTN,AMRC,AAPB,AAN
1,AIN,BFAM,AAON,ACHV,IAGG,ACIU,AAON,ACNB,THS,BFS,ABM,AAON
2,AIT,CPS,ACA,ACRE,MDGL,AEVA,AAT,ACRE,NaN,CGON,AC,AAT
3,AKYA,EOLS,ACCO,ACRS,MTBA,AGYS,ABG,ADVM,NaN,CRBP,ACEL,ABCB
4,ALE,FBL,ACLX,AGGH,MYPS,AIN,ABOS,AFCG,NaN,EMBC,ACHV,ABG
...,...,...,...,...,...,...,...,...,...,...,...,...
1231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZIP
1232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZUMZ
1233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZURA
1234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZVRA


In [ ]:
df_count(df_5)

금_소_양_5 개수: 168
금_소_음_5 개수: 21
금_대_양_5 개수: 365
금_대_음_5 개수: 105
코인_소_양_5 개수: 7
코인_소_음_5 개수: 196
코인_대_양_5 개수: 844
코인_대_음_5 개수: 267
환율_소_양_5 개수: 2
환율_소_음_5 개수: 25
환율_대_양_5 개수: 390
환율_대_음_5 개수: 1236


# 상관계수 0.7

In [ ]:
add_corr(gold_df, '금', pearson_threshold=0.7, spearman_threshold=0.7)
add_corr(coin_df, '코인', pearson_threshold=0.7, spearman_threshold=0.7)
add_corr(exchange_df, '환율', pearson_threshold=0.7, spearman_threshold=0.7)

# 결과 확인
print(gold_df)
print(coin_df)
print(exchange_df)

     tck_iem_cd   금_소_피어슨  금_소_스피어만   금_대_피어슨  금_대_스피어만  금_소_양_상관관계  \
AAN         AAN -0.661378 -0.728040  0.572818       0.8           0   
AAON       AAON  0.177372  0.300000  0.561124       0.8           0   
AAPB       AAPB -0.273125 -0.316667  0.880535       0.2           0   
AAT         AAT  0.127686  0.283333 -0.085943       0.2           0   
ABCB       ABCB  0.530589  0.400000  0.190518       0.2           0   
...         ...       ...       ...       ...       ...         ...   
ZUMZ       ZUMZ -0.062324 -0.166667  0.244698       0.4           0   
ZURA       ZURA  0.073128  0.050000  0.464929       0.2           0   
ZVRA       ZVRA  0.599648  0.416667 -0.643747      -0.4           0   
ZYME       ZYME -0.028913  0.100000 -0.387906       0.0           0   
ZYXI       ZYXI -0.093530 -0.066667 -0.362097       0.2           0   

      금_소_음_상관관계  금_대_양_상관관계  금_대_음_상관관계  
AAN            0           0           0  
AAON           0           0           0  
AAPB           0  

In [ ]:
# 상관관계 분석할 데이터프레임 목록
dataframes = {'금': gold_df, '코인': coin_df, '환율': exchange_df}

# 결과 저장을 위한 딕셔너리
related_names_all_7 = {}

# 각 데이터프레임에 대해 반복하여 관련 종목 리스트 생성
for key, df in dataframes.items():
    related_names_7 = get_related_names(df, key, threshold=7)
    related_names_all_7.update(related_names_7)  # 7의 결과를 업데이트

# 결과를 데이터프레임으로 변환
df_7 = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in related_names_all_7.items()]))
df_7

,금_소_양_7,금_소_음_7,금_대_양_7,금_대_음_7,코인_소_양_7,코인_소_음_7,코인_대_양_7,코인_대_음_7,환율_소_양_7,환율_소_음_7,환율_대_양_7,환율_대_음_7
0,ALEC,MELI,ADEA,ACHV,IAGG,AGYS,AAN,ADVM,NaN,FENC,AAPB,AAN
1,ANIK,PYPY,AFRI,AGGH,NaN,AMR,AAON,AFCG,NaN,NaN,ABM,AAON
2,CSGS,RACE,AGYS,AIRS,NaN,ATSG,ABG,AGR,NaN,NaN,AC,AAT
3,EEX,TFX,AIT,AKRO,NaN,BAP,ACEL,AILE,NaN,NaN,ACEL,ABCB
4,EMBC,NaN,AIYY,ALTG,NaN,CNSL,ACHV,AISP,NaN,NaN,ACHV,ABG
...,...,...,...,...,...,...,...,...,...,...,...,...
1231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZIP
1232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZUMZ
1233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZURA
1234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZVRA


In [ ]:
df_count(df_7)

금_소_양_7 개수: 19
금_소_음_7 개수: 4
금_대_양_7 개수: 233
금_대_음_7 개수: 68
코인_소_양_7 개수: 1
코인_소_음_7 개수: 24
코인_대_양_7 개수: 388
코인_대_음_7 개수: 146
환율_소_양_7 개수: 0
환율_소_음_7 개수: 1
환율_대_양_7 개수: 390
환율_대_음_7 개수: 1236


# 최종 데이터프레임 정리

In [ ]:
final_corr_df = pd.concat([df_3, df_5, df_7], axis=1)
final_corr_df

,금_소_양_3,금_소_음_3,금_대_양_3,금_대_음_3,코인_소_양_3,코인_소_음_3,코인_대_양_3,코인_대_음_3,환율_소_양_3,환율_소_음_3,...,금_대_양_7,금_대_음_7,코인_소_양_7,코인_소_음_7,코인_대_양_7,코인_대_음_7,환율_소_양_7,환율_소_음_7,환율_대_양_7,환율_대_음_7
0,ABCB,AAN,AAN,ABM,ACRS,ABCB,AAN,ABM,AFRI,ABG,...,ADEA,ACHV,IAGG,AGYS,AAN,ADVM,NaN,FENC,AAPB,AAN
1,ABG,AGGH,AAON,ACHV,AGGH,ABG,AAON,AC,AMPL,ABOS,...,AFRI,AGGH,NaN,AMR,AAON,AFCG,NaN,NaN,ABM,AAON
2,ABOS,AGR,AC,ACRE,AILE,ACEL,AAT,ACNB,BRC,ACA,...,AGYS,AIRS,NaN,ATSG,ABG,AGR,NaN,NaN,AC,AAT
3,AC,AMDY,ACA,ACRS,BUCK,ACIU,ABG,ACRE,CTRI,ACLS,...,AIT,AKRO,NaN,BAP,ACEL,AILE,NaN,NaN,ACEL,ABCB
4,ACA,APLY,ACCO,AGEN,CCRN,ACLS,ABOS,ADVM,ENV,ACLX,...,AIYY,ALTG,NaN,CNSL,ACHV,AISP,NaN,NaN,ACHV,ABG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZIP
1232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZUMZ
1233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZURA
1234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZVRA


In [ ]:
df_count(final_corr_df)

금_소_양_3 개수: 560
금_소_음_3 개수: 100
금_대_양_3 개수: 595
금_대_음_3 개수: 256
코인_소_양_3 개수: 39
코인_소_음_3 개수: 634
코인_대_양_3 개수: 988
코인_대_음_3 개수: 345
환율_소_양_3 개수: 26
환율_소_음_3 개수: 206
환율_대_양_3 개수: 390
환율_대_음_3 개수: 1236
금_소_양_5 개수: 168
금_소_음_5 개수: 21
금_대_양_5 개수: 365
금_대_음_5 개수: 105
코인_소_양_5 개수: 7
코인_소_음_5 개수: 196
코인_대_양_5 개수: 844
코인_대_음_5 개수: 267
환율_소_양_5 개수: 2
환율_소_음_5 개수: 25
환율_대_양_5 개수: 390
환율_대_음_5 개수: 1236
금_소_양_7 개수: 19
금_소_음_7 개수: 4
금_대_양_7 개수: 233
금_대_음_7 개수: 68
코인_소_양_7 개수: 1
코인_소_음_7 개수: 24
코인_대_양_7 개수: 388
코인_대_음_7 개수: 146
환율_소_양_7 개수: 0
환율_소_음_7 개수: 1
환율_대_양_7 개수: 390
환율_대_음_7 개수: 1236


In [ ]:
final_corr_df.to_csv('상관관계_대소폭_결과.csv', index=False, encoding='utf-8-sig')

# 전체 ETF와 대소폭 ETF 교집합

In [ ]:
all_etf = pd.read_csv('/content/gdrive/MyDrive/NH 공모전/data/상관관계_ETF_결과.csv')
hl_etf = pd.read_csv('/content/gdrive/MyDrive/NH 공모전/data/상관관계_대소폭_결과.csv')

In [ ]:
print(hl_etf.columns)
print(all_etf.columns)

Index(['금_소_양_3', '금_소_음_3', '금_대_양_3', '금_대_음_3', '코인_소_양_3', '코인_소_음_3',
       '코인_대_양_3', '코인_대_음_3', '환율_소_양_3', '환율_소_음_3', '환율_대_양_3', '환율_대_음_3',
       '금_소_양_5', '금_소_음_5', '금_대_양_5', '금_대_음_5', '코인_소_양_5', '코인_소_음_5',
       '코인_대_양_5', '코인_대_음_5', '환율_소_양_5', '환율_소_음_5', '환율_대_양_5', '환율_대_음_5',
       '금_소_양_7', '금_소_음_7', '금_대_양_7', '금_대_음_7', '코인_소_양_7', '코인_소_음_7',
       '코인_대_양_7', '코인_대_음_7', '환율_소_양_7', '환율_소_음_7', '환율_대_양_7', '환율_대_음_7'],
      dtype='object')
Index(['금_양_3', '금_음_3', '코인_양_3', '코인_음_3', '환율_양_3', '환율_음_3', '금_양_5',
       '금_음_5', '코인_양_5', '코인_음_5', '환율_양_5', '환율_음_5', '금_양_5.1', '금_음_5.1',
       '코인_양_5.1', '코인_음_5.1', '환율_양_5.1', '환율_음_5.1', '금_양_7', '금_음_7',
       '코인_양_7', '코인_음_7', '환율_양_7', '환율_음_7'],
      dtype='object')


In [ ]:
df_count(all_etf)

금_양_3 개수: 476
금_음_3 개수: 50
코인_양_3 개수: 76
코인_음_3 개수: 291
환율_양_3 개수: 8
환율_음_3 개수: 283
금_양_5 개수: 95
금_음_5 개수: 8
코인_양_5 개수: 9
코인_음_5 개수: 57
환율_양_5 개수: 1
환율_음_5 개수: 36
금_양_5.1 개수: 95
금_음_5.1 개수: 8
코인_양_5.1 개수: 9
코인_음_5.1 개수: 57
환율_양_5.1 개수: 1
환율_음_5.1 개수: 36
금_양_7 개수: 3
금_음_7 개수: 0
코인_양_7 개수: 0
코인_음_7 개수: 3
환율_양_7 개수: 0
환율_음_7 개수: 0


In [ ]:
def calculate_intersection(df1, col1, df2, col2):
    set1 = set(df1[col1])
    set2 = set(df2[col2])
    return set1.intersection(set2)

# 금 양수 교집합 계산
inter_low_pos = calculate_intersection(hl_etf, '금_소_양_3', all_etf, '금_양_3')
inter_high_pos = calculate_intersection(hl_etf, '금_대_양_3', all_etf, '금_양_3')

# 두 교집합 결과를 합치고 중복 제거
inter_gold_pos_3 = inter_low_pos.union(inter_high_pos)

# 결과 출력
print("inter_gold_pos_3 결과:", inter_gold_pos_3)
print("inter_gold_pos_3 개수:", len(inter_gold_pos_3))

# 금 음수 교집합 계산
inter_low_neg = calculate_intersection(hl_etf, '금_소_음_3', all_etf, '금_음_3')
inter_high_neg = calculate_intersection(hl_etf, '금_대_음_3', all_etf, '금_음_3')

# 두 교집합 결과를 합치고 중복 제거
inter_gold_neg_3 = inter_low_neg.union(inter_high_neg)

# 결과 출력
print("inter_gold_neg_3 결과:", inter_gold_neg_3)
print("inter_gold_neg_3 개수:", len(inter_gold_neg_3))

inter_gold_pos_3 결과: {'LCTX', 'DNTH', 'ZG', 'ATNM', 'NBR', 'CWCO', 'AGYS', 'NL', 'BLK', 'THFF', 'ATSG', 'MOND', 'SASR', 'CRBP', 'ENSG', 'KROS', 'ULH', 'WIX', 'SFNC', 'AVAV', 'CBU', 'ABOS', 'DCI', 'MLTX', 'ACA', 'VALU', 'TPC', 'BWIN', 'GRND', 'PRTH', 'SLRN', 'KGS', 'PROK', 'CHDN', 'NTGR', 'EFSC', 'HIPO', 'BOH', 'MTRN', 'AXON', 'SWI', 'REPL', 'NBTB', 'VUZI', 'PAYS', 'WHD', 'DRS', 'HYLN', 'OEC', 'UHT', 'DCGO', 'WSBC', 'KEX', 'SFST', 'NARI', 'SEZL', 'NECB', 'INSE', 'BZH', 'FISI', 'FRHC', 'CCOI', 'THRD', 'AIN', 'LEGH', 'UNF', 'CTOS', 'OPFI', 'FICO', 'AAON', 'RLGT', 'AIYY', 'HWC', 'PB', 'FRPH', 'GABC', 'JACK', 'HLI', 'SYBT', 'HDSN', 'LSPD', 'PPBI', 'TCMD', 'EMBC', 'EPAC', 'STEL', 'NWS', 'MMYT', 'VMI', 'WD', 'AN', 'BLFS', 'UTHR', 'HBNC', 'CMPO', 'MCRI', 'PWP', 'PETQ', 'CIX', 'LPRO', 'FLWS', 'HIW', 'IMMR', 'CNS', 'LINC', 'PLUS', 'NVMI', 'MIRM', 'CLAR', 'BOOT', 'RMNI', 'LBPH', 'XHR', 'BUSE', 'REPX', 'OSK', 'GOOD', 'STRL', 'HCSG', 'ESE', 'GBCI', 'CDRE', 'STGW', 'NABL', 'SPIR', 'GBTG', 'CEIX', 'M

In [ ]:
def calculate_intersection(df1, col1, df2, col2):
    set1 = set(df1[col1].dropna())  # NaN 제거
    set2 = set(df2[col2].dropna())  # NaN 제거
    return set1.intersection(set2)

# 금, 코인, 환율을 위한 열 이름 목록
types = ['금', '코인', '환율']
suffixes = ['3', '5', '7']

# 결과를 저장할 딕셔너리
results = {}

for t in types:
    for s in suffixes:
        # 양수 교집합 계산
        inter_low_pos = calculate_intersection(hl_etf, f'{t}_소_양_{s}', all_etf, f'{t}_양_{s}')
        inter_high_pos = calculate_intersection(hl_etf, f'{t}_대_양_{s}', all_etf, f'{t}_양_{s}')
        inter_gold_pos = inter_low_pos.union(inter_high_pos)

        results[f'inter_{t}_pos_{s}'] = inter_gold_pos
        # print(f"{t} 양수 {s} 교집합 결과:", inter_gold_pos)
        print(f"{t} 양수 {s} 교집합 개수:", len(inter_gold_pos))

        # 음수 교집합 계산
        inter_low_neg = calculate_intersection(hl_etf, f'{t}_소_음_{s}', all_etf, f'{t}_음_{s}')
        inter_high_neg = calculate_intersection(hl_etf, f'{t}_대_음_{s}', all_etf, f'{t}_음_{s}')
        inter_gold_neg = inter_low_neg.union(inter_high_neg)

        results[f'inter_{t}_neg_{s}'] = inter_gold_neg
        # print(f"{t} 음수 {s} 교집합 결과:", inter_gold_neg)
        print(f"{t} 음수 {s} 교집합 개수:", len(inter_gold_neg))
        # print()

금 양수 3 교집합 개수: 410
금 음수 3 교집합 개수: 38
금 양수 5 교집합 개수: 68
금 음수 5 교집합 개수: 6
금 양수 7 교집합 개수: 3
금 음수 7 교집합 개수: 0
코인 양수 3 교집합 개수: 50
코인 음수 3 교집합 개수: 222
코인 양수 5 교집합 개수: 4
코인 음수 5 교집합 개수: 37
코인 양수 7 교집합 개수: 0
코인 음수 7 교집합 개수: 2
환율 양수 3 교집합 개수: 7
환율 음수 3 교집합 개수: 252
환율 양수 5 교집합 개수: 1
환율 음수 5 교집합 개수: 28
환율 양수 7 교집합 개수: 0
환율 음수 7 교집합 개수: 0


In [ ]:
# 결과를 데이터프레임으로 변환
data = {}

for key, value in results.items():
    data[key] = list(value)  # 교집합 결과를 리스트로 변환

# 데이터프레임 생성
df_results = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in data.items()]))

# CSV 파일로 저장
df_results.to_csv('최종_교집합_ETF.csv', index=False, encoding='utf-8-sig')

print("결과를 '최종_교집합_ETF.csv'로 저장했습니다.")

결과를 '최종_교집합_ETF.csv'로 저장했습니다.
